<a href="https://colab.research.google.com/github/Sunkohli/Lab-1-Quantum-Algorithm/blob/main/DJ_algo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:

!pip install qiskit qiskit_aer qiskit-ibm-runtime matplotlib pylatexenc

In [3]:
%pip install "qiskit-aer>=0.17.0"

In [4]:
###DJ algo
#import required packages
import qiskit
from qiskit import QuantumRegister, QuantumCircuit
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler
from qiskit import transpile
from qiskit_aer import Aer, AerSimulator
import numpy as np
from qiskit.visualization import plot_histogram

In [5]:
def dj_query(num_qubits):
    # Create a circuit implementing for a query gate for a random function
    # satisfying the promise for the Deutsch-Jozsa problem.

    qc = QuantumCircuit(num_qubits + 1)

    if np.random.randint(0, 2):
        # Flip output qubit with 50% chance
        qc.x(num_qubits)
    if np.random.randint(0, 2):
        # return constant circuit with 50% chance
        return qc

    # Choose half the possible input strings
    on_states = np.random.choice(
        range(2**num_qubits),  # numbers to sample from
        2**num_qubits // 2,  # number of samples
        replace=False,  # makes sure states are only sampled once
    )

    def add_cx(qc, bit_string):
        for qubit, bit in enumerate(reversed(bit_string)):
            if bit == "1":
                qc.x(qubit)
        return qc

    for state in on_states:
        qc.barrier()  # Barriers are added to help visualize how the functions are created.
        qc = add_cx(qc, f"{state:0b}")
        qc.mcx(list(range(num_qubits)), num_qubits)
        qc = add_cx(qc, f"{state:0b}")

    qc.barrier()

    return qc

In [6]:
def compile_circuit(function: QuantumCircuit):
    # Compiles a circuit for use in the Deutsch-Jozsa algorithm.

    n = function.num_qubits - 1
    qc = QuantumCircuit(n + 1, n)
    qc.x(n)
    qc.h(range(n + 1))
    qc.compose(function, inplace=True)
    qc.h(range(n))
    qc.measure(range(n), range(n))
    return qc

In [7]:
def dj_algorithm(function: QuantumCircuit):
    # Determine if a function is constant or balanced.
    qc = compile_circuit(function)
    backend = AerSimulator()
    qc_compiled = transpile(qc, backend)
    job_sim = backend.run(qc, shots=32,memory=True)
    result_sim = job_sim.result()
    measurements = result_sim.get_memory()
    print(measurements)
    if measurements[0] == "0":
        return "constant"
    return "balanced"


In [24]:
f = dj_query(2)
display(f.draw())
display(dj_algorithm(f))

q_0: ─────
          
q_1: ─────
     ┌───┐
q_2: ┤ X ├
     └───┘

['00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00']


'balanced'